In [2]:
import wave
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import sklearn as sk

import scipy.io
import librosa
import librosa.display
%matplotlib inline
import pandas as pd


import tensorflow as tf


In [3]:
def load_sound_files(file_paths):
    raw_sounds = []
    
    
    items = os.listdir(file_paths)

    
    #searches through the input file for any files 
    #named .wav and adds them to the list
    
    newlist = []
    for names in items:
        if names.endswith(".wav"):
            newlist.append(names)
   
    #Loads the files found above in with librosa
    for fp in newlist:
        fp = os.path.join(path, fp)
        X,sr = librosa.load(fp,500)  
        raw_sounds.append(X)
    return raw_sounds



In [23]:
path = '/home/tim/Documents/Masters/Data/93-001-2321.ch13/'
raw_sounds =  load_sound_files(path)

In [24]:
"""
For testing purposes at the moment, code is here to allow you to view the input data.


"""
def plot_waves(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        librosa.display.waveplot(np.array(f),sr=500)
        plt.title(n.title())
        i += 1
    plt.suptitle('Figure 1: Waveplot',x=0.5, y=0.915,fontsize=18)
    plt.show()
    
def plot_specgram(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        f_ = librosa.stft(y = f, n_fft= 256, win_length= 256)
        librosa.display.specshow(librosa.amplitude_to_db(f_),
                                        sr = 500,
                                        y_axis = 'log',
                                        hop_length = 64 )
        #specgram(np.array(f), Fs=500, mode = 'psd')
        plt.title(n.title())
        #plt.colorbar(format='%+4.0f dB')
        i += 1
    plt.suptitle('Figure 2: Spectrogram',x=0.5, y=0.915,fontsize=18)
    plt.show()

def plot_log_power_specgram(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi = 1200)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        D = librosa.logamplitude(np.abs(librosa.stft(f))**2, ref_power=np.max)
        librosa.display.specshow(D,x_axis='time' ,y_axis='log', sr = 500)
        plt.title(n.title())
        i += 1
    plt.suptitle('Figure 3: Log power spectrogram',x=0.5, y=0.915,fontsize=18)
    plt.show()
 
plot_waves('minke',raw_sounds)
plot_specgram('minke',raw_sounds)
plot_log_power_specgram('minke',raw_sounds)

In [6]:
def xtract_features(file_name):
    X, sample_rate = librosa.load(file_name)
    # stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    # tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return X, sample_rate, mfccs#chroma,mel,contrast,tonnetz


In [28]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += int(window_size / 2)
    return start 

In [29]:
  


def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 20, frames = 41):
    window_size = 512 * (frames - 1)

    mfccs = []
    labels = []

    for l, sub_dir in enumerate(sub_dirs):
        
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            
            sound_clip, s = librosa.load(fn)
            label = l
            start = windows(sound_clip, window_size)
            
            

            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]
                mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                    
            mfccs.append(mfcc)
            labels.append(l)


    features = np.asarray(mfccs).reshape(len(mfccs),frames,bands)
    return features, np.array(labels,dtype = np.int)

In [30]:
def get_directories(ROOT_PATH, directory):

    directories = [d for d in os.listdir(directory) 
                if os.path.isdir(os.path.join(directory, d))]
    print(directories)
    return directories

In [31]:

ROOT_PATH = "/home/tim/Documents/Masters/Data"
train_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Training")
test_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Testing")

train_directories = get_directories(ROOT_PATH, train_data_directory)
test_directories = get_directories(ROOT_PATH, test_data_directory)

#tr_features, tr_labels = load_data(train_data_directory)
#ts_features, ts_labels = load_data(train_data_directory)

tr_features, tr_labels = extract_features(train_data_directory, train_directories)
#ts_features, ts_labels = extract_features(train_data_directory, test_directories)


['Training', 'Noise', 'Minke']
['001', '000']


NameError: name 'end' is not defined

In [94]:

pd.DataFrame(tr_features)

ValueError: Must pass 2-d input

In [11]:
def one_hot_encode(labels):
    n_labels = len(labels)
    print('labels ', n_labels)
    n_unique_labels = len(np.unique(labels))
    print('unique labels ', n_unique_labels)
    one_hot_encode = np.eye(n_unique_labels)
    print('one Hot', one_hot_encode)
    
    return one_hot_encode

In [14]:
# This stuff needs to be moved from above to clean up the code. 
# ROOT_PATH = "/home/tim/Documents/Masters/Data"
# train_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Training")
# test_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Testing")

# tr_features, tr_labels = load_data(train_data_directory)
# ts_features, ts_labels = load_data(train_data_directory)

tr_labels = one_hot_encode(tr_labels)
#ts_labels = one_hot_encode(ts_labels)
print(tr_labels)

NameError: name 'tr_labels' is not defined

In [13]:
tf.reset_default_graph()

learning_rate = 0.01
training_iters = 1000
batch_size = 50
display_step = 200

# Network Parameters
n_input = 20
number_of_layers = 2
n_steps = 41
n_hidden = 300
n_classes = 2 

x = tf.placeholder("float", [None, n_steps, n_input], name= 'x')
y = tf.placeholder("float", [None, n_classes], name = 'y')

weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

In [70]:
def lstm_cell(n_hidden,state_is_tuple = True):
  return tf.contrib.rnn.BasicLSTMCell(n_hidden)

In [71]:
def RNN(x, weight, bias, number_of_layers):
    
    cell = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell(n_hidden,state_is_tuple = True) for _ in range(number_of_layers)])
    output, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
    output = tf.transpose(output, [1, 0, 2])
    last = tf.gather(output, int(output.get_shape()[0]) - 1)
    return tf.nn.softmax(tf.matmul(last, weight) + bias)




In [72]:
prediction = RNN(x, weight, bias, number_of_layers)

# Define loss and optimizer
loss_f = -tf.reduce_sum(y * tf.log(prediction))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_f)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

/home/tim/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [73]:

with tf.Session() as session:
    session.run(init)
    
    for itr in range(training_iters):    
        offset = (itr *  batch_size) % (tr_labels.shape[0] - batch_size)
        batch_x = tr_features[None, offset:(offset + batch_size), :]
        print('length of batch_x' , len(batch_x))
        batch_y = tr_labels[offset:(offset + batch_size), :]
        print('length of y' , len(batch_y))
        _, c = session.run([optimizer, loss_f],feed_dict={x: batch_x, y : batch_y})
            
        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(itr) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    
    print('Test accuracy: ',round(session.run(accuracwith tf.Session() as session:
    session.run(init)
    
    for itr in range(training_iters):    
        offset = (itr *  batch_size) % (tr_labels.shape[0] - batch_size)
        batch_x = tr_features[None, offset:(offset + batch_size), :]
        print('length of batch_x' , len(batch_x))
        batch_y = tr_labels[offset:(offset + batch_size), :]
        print('length of y' , len(batch_y))
        _, c = session.run([optimizer, loss_f],feed_dict={x: batch_x, y : batch_y})
            
        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(itr) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    y, feed_dict={x: ts_features, y: ts_labels}) , 3))

length of batch_x 1
length of y 2


ValueError: Cannot feed value of shape (1, 50, 41, 20) for Tensor 'x:0', which has shape '(?, 41, 20)'

In [75]:
with tf.Session() as session:
    session.run(init)
    
    for itr in range(training_iters):    
        offset = (itr * batch_size) % (tr_labels.shape[0] - batch_size)
        batch_x = tr_features[offset:(offset + batch_size), :, :]
        batch_x = np.squeeze(batch_x)
        batch_y = tr_labels[offset:(offset + batch_size), :]
        _, c = session.run([optimizer, loss_f],feed_dict={x: batch_x, y : batch_y})
            
        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
    
    print('Test accuracy: ',round(session.run(accuracy, feed_dict={x: ts_features, y: ts_labels}) , 3))

InvalidArgumentError: Incompatible shapes: [2,2] vs. [50,2]
	 [[Node: mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_y_0_1, Log)]]

Caused by op 'mul', defined at:
  File "/home/tim/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tim/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tim/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/tim/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/tim/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/tim/anaconda3/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/home/tim/anaconda3/lib/python3.5/site-package